# ReAct Agent

In [1]:
import logging
import os
import openai
import itertools

from typing import List

from actionweaver.llms.openai.chat import OpenAIChatCompletion
from actionweaver.llms.openai.tokens import TokenUsageTracker
from actionweaver import action, ActionHandlerMixin, SelectOne, RequireNext

from actionweaver.mixins.examples import LangChainTools, Folium, OpenAIAPI
openai.api_key = os.getenv("OPENAI_API_KEY")

ImportError: cannot import name 'ActionHandlerMixin' from 'actionweaver' (/Users/tenghu/Code/actiontools/actionweaver/__init__.py)

In [ ]:
logging.basicConfig(
    filename='react.log',
    filemode='a',
    format='%(asctime)s.%(msecs)04d %(levelname)s {%(module)s} [%(funcName)s] %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


logger.info("dsf")

In [2]:
class LLMAgent(ActionHandlerMixin):
    def __init__(self, logger):
        self.logger = logger
        self.token_tracker = TokenUsageTracker(budget=None, logger=logger)
        self.llm = OpenAIChatCompletion("gpt-3.5-turbo", token_usage_tracker = self.token_tracker, logger=logger)
        
        self.messages = [{"role": "system", "content": "You are a resourceful assistant, able to inquire for more information if you cannot provide a confident answer to a question."}]
        self.buffer = [] 
    
    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]

        self.buffer = self.messages[-6:] # We only take last 6 messages for input to llm
        len_buffer = len(self.buffer)
        
        response = self.llm.create(messages=self.buffer, stream=True)

        self.messages = self.messages[: len(self.messages) - len_buffer] + self.buffer
        return response

    
class AgentV0(LLMAgent, LangChainTools, Folium, OpenAIAPI):
    @action(name="Planning")
    def plan(self, steps: List[str]) -> str:
        """
        Invoke this function to create a list of tasks.
        """
        concatenated_steps = ""
        for index, task in enumerate(steps):
            concatenated_steps += f"{index}: {task}\n"
        
        return concatenated_steps

def print_output(output):
    if type(output) == itertools._tee:
        for chunk in output:
            content = chunk["choices"][0].get("delta", {}).get("content")
            if content is not None:
                print(content, end='')
    else:
        print (output)


agent = AgentV0(logger)

NameError: name 'ActionHandlerMixin' is not defined

In [3]:
print_output(agent("task: showing top 10 chinese cities based on population.\n Create a plan finish the task, each step ONLY use one of tools available to you"))

NameError: name 'print_output' is not defined

In [20]:
agent._action_handlers.name_to_action

{'GoogleSearch': <actionweaver.actions.action.Action at 0x10bfcf100>,
 'ShowMap': <actionweaver.actions.action.Action at 0x10bb1d370>,
 'GenerateImage': <actionweaver.actions.action.Action at 0x10bfcfca0>,
 'Planning': <actionweaver.actions.action.Action at 0x1220bc9a0>}